In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_classifier import DNNClassifer
from random import randint
from visualizer import *

2023-01-15 16:35:21.491519: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-15 16:35:21.491622: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-15 16:35:21.491635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
granular_data = pd.read_csv("../data/BTCUSDT-15m.csv", parse_dates=["Open Time"], index_col = "Open Time")

In [3]:
def resolve_conflict_tp_sl(r,i,take_profit_rate,stop_loss_rate, timeframe_in_ms = 24*60*60*1000):

    row = r.copy()
   
    if (row["ol_tp_cond_{}".format(i)] & row["ol_sl_cond_{}".format(i)]):
    
        start_check_time = int(row.name) + i * timeframe_in_ms
        end_check_time = start_check_time + timeframe_in_ms

       
        lookup_data = granular_data.iloc[
            (granular_data.index>start_check_time) & 
            (granular_data.index<end_check_time)].copy()

        lookup_data["long_take_profit"] = row["Close"]*(1+take_profit_rate)
        lookup_data["long_stop_loss"] = row["Close"]*(1-stop_loss_rate)
        lookup_data["ol_tp_cond"] = lookup_data["High"] >= lookup_data["long_take_profit"]
        lookup_data["ol_sl_cond"] = lookup_data["Low"] <= lookup_data["long_stop_loss"]

        tp_hit_first_index = lookup_data.loc[lookup_data["ol_tp_cond"]].index.min()
        sl_hit_first_index = lookup_data.loc[lookup_data["ol_sl_cond"]].index.min()    

        if tp_hit_first_index < sl_hit_first_index:
            outcome = 1
        elif tp_hit_first_index > sl_hit_first_index:
            outcome = -1
        else:
            outcome = 0

        return outcome


In [4]:
take_profit_rate = 0.01
stop_loss_rate = 0.01
max_duration = 7

data["long_take_profit"] = data["Close"]*(1+take_profit_rate)
data["long_stop_loss"] = data["Close"]*(1-stop_loss_rate)

for i in range(1,max_duration+1):
    
    data["High_forward_{}".format(i)] = data["High"].shift(-i)
    data["Low_forward_{}".format(i)] = data["Low"].shift(-i)
    data["ol_tp_cond_{}".format(i)] = data["High_forward_{}".format(i)] >= data["long_take_profit"]
    data["ol_sl_cond_{}".format(i)] = data["Low_forward_{}".format(i)] <= data["long_stop_loss"]
    data["resolved_{}".format(i)] = data.apply(lambda row: resolve_conflict_tp_sl(
        r=row,
        i=i,
        take_profit_rate=take_profit_rate,
        stop_loss_rate=stop_loss_rate
        ),
        axis=1
    )

data.dropna(inplace=True)

    

In [5]:
data.head(5)

,Open,High,Low,Close,Volumn,Close Time,Quote Asset Volumn,Number of Trades,Taker Buy Base Asset Volumn,Taker Buy Quote Asset Volumn,...,High_forward_6,Low_forward_6,ol_tp_cond_6,ol_sl_cond_6,resolved_6,High_forward_7,Low_forward_7,ol_tp_cond_7,ol_sl_cond_7,resolved_7
Open Time,,,,,,,,,,,,,,,,,,,,,
1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06,...,4265.80,4013.89,False,True,2,4371.68,4085.01,True,True,-1
1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264,1503100799999,5.086958e+06,5233,972.868710,4.129123e+06,...,4371.68,4085.01,True,False,2,4453.91,4247.48,True,False,2
1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763,1503187199999,1.549484e+06,2153,274.336042,1.118002e+06,...,4453.91,4247.48,True,False,2,4367.00,4212.41,True,False,2
1503187200000,4120.98,4211.08,4032.62,4086.29,467.083022,1503273599999,1.930364e+06,2321,376.795947,1.557401e+06,...,4367.00,4212.41,True,False,2,4400.00,4285.54,True,False,2
1503273600000,4069.13,4119.62,3911.79,4016.00,691.743060,1503359999999,2.797232e+06,3972,557.356107,2.255663e+06,...,4400.00,4285.54,True,False,2,4399.82,4124.54,True,False,2
